# opencv

## Creation of Simple Paint canva application Project. 
- Using Trackbar
  - Red color
  - Blue color
  - Green color
  - Thickness
- Shapes
  - Rectangle
  - Circle
  - Line
- Buttons
  - Pen
  - Eraser
  - Save
  - Delete
  - Quit

In [ ]:
import cv2
import numpy as np

# creation of background
white = np.full((600, 1200, 3), 255, dtype=np.uint8)
cv2.namedWindow("canva", cv2.WINDOW_FREERATIO)  # creating a namedWindow "canva"


def f1(x):  # reference function to trackbar
    pass


# Creation of Trackbars to adjust the intensities of the colors.
cv2.createTrackbar("blue", "canva", 0, 255, f1)
cv2.createTrackbar("green", "canva", 0, 255, f1)
cv2.createTrackbar("red", "canva", 0, 255, f1)
cv2.createTrackbar("thickness", "canva", 1, 100, f1)


def trac():  # function to get the trackbars values.
    bl = cv2.getTrackbarPos("blue", "canva")
    gr = cv2.getTrackbarPos("green", "canva")
    re = cv2.getTrackbarPos("red", "canva")
    th = cv2.getTrackbarPos("thickness", "canva")
    return [(bl, gr, re), th]


x1, y1 = None, None
drawing = False
mode = "Pen"
shape_preview = None

# Shapes positions on sidebar (right side.)
shapes = {
    "Rectangle": (1070, 70, 1130, 110),  # rectangle outline
    "Circle": (1100, 160, 30),           # circle center + radius
    "Line": ((1070, 220), (1130, 260))   # line endpoints
}

canva_bound = 1050  # to ensure that drawing area doesn't include the shapes area.


# Function to track the mouse actions.
def f2(event, x, y, flags, user):
    global x1, y1, drawing, mode, shape_preview, canva_bound

    if event == cv2.EVENT_LBUTTONDOWN:  # tracks mouse left click
        # check whether click is inside the area of shapes
        Rx1, Ry1, Rx2, Ry2 = shapes["Rectangle"]
        Cx, Cy, r = shapes["Circle"]
        (Lx1, Ly1), (Lx2, Ly2) = shapes["Line"]

        # assigning the mode value with respect to click position
        if Rx1 <= x <= Rx2 and Ry1 <= y <= Ry2:
            mode = "Rectangle"
        elif (x - Cx) ** 2 + (y - Cy) ** 2 <= r ** 2:
            mode = "Circle"
        elif Lx1 <= x <= Lx2 and Ly1 <= y <= Ly2:
            mode = "Line"

        # if click is inside the canva expect shapes area.
        elif x <= canva_bound:
            drawing = True  # if click inside the canva then only drawing is activated
            x1 = x
            y1 = y
            shape_preview = white.copy()  # to track the drawing preview

    elif event == cv2.EVENT_MOUSEMOVE and drawing:  # mouse move after click
        shape_preview = white.copy()
        color, thick = trac()

        if mode == "Pen":
            cv2.line(white, (x1, y1), (x, y), color, thick)  # directly on white background
            x1, y1 = x, y

        elif mode == "Eraser":
            cv2.line(white, (x1, y1), (x, y), (255, 255, 255), thick)  # white color eraser
            x1, y1 = x, y

        elif mode == "Rectangle":
            cv2.rectangle(shape_preview, (x1, y1), (x, y), color, thick)  # preview

        elif mode == "Circle":
            radius = int(((x - x1) ** 2 + (y - y1) ** 2) ** 0.5)
            cv2.circle(shape_preview, (x1, y1), radius, color, thick)  # preview

        elif mode == "Line":
            cv2.line(shape_preview, (x1, y1), (x, y), color, thick)  # preview

    elif event == cv2.EVENT_LBUTTONUP and drawing:  # left click released
        color, thick = trac()

        if mode == "Rectangle":
            cv2.rectangle(white, (x1, y1), (x, y), color, thick)

        elif mode == "Circle":
            radius = int(((x - x1) ** 2 + (y - y1) ** 2) ** 0.5)
            cv2.circle(white, (x1, y1), radius, color, thick)

        elif mode == "Line":
            cv2.line(white, (x1, y1), (x, y), color, thick)

        drawing = False
        x1, y1 = None, None
        shape_preview = None


cv2.setMouseCallback("canva", f2)  # to call f2 and track the mouse actions

i = 1
while True:
    display = white.copy() if shape_preview is None else shape_preview.copy()

    # Displaying the options to the user
    cv2.putText(display, "MODES = {Q: Quit | P: Pen | E: Eraser | S: Save | D: Delete}",
                (10, 540), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 0), 2)
    cv2.putText(display, f"MODE = {mode}", (10, 580),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 150, 0), 2)

    # Displaying the shapes on the sidebar of canva
    Rx1, Ry1, Rx2, Ry2 = shapes["Rectangle"]
    cv2.putText(display, "Shapes", (Rx1, Ry1 - 20),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.rectangle(display, (Rx1, Ry1), (Rx2, Ry2), (0, 0, 0), 2)
    Cx1, Cx2, r = shapes["Circle"]
    cv2.circle(display, (Cx1, Cx2), r, (0, 0, 0), 2)
    (Lx1, Ly1), (Lx2, Ly2) = shapes["Line"]
    cv2.line(display, (Lx1, Ly1), (Lx2, Ly2), (0, 0, 0), 2)

    # Linking the namedWindow canva to display
    cv2.imshow("canva", display)
    key = cv2.waitKey(1)

    # Modes to perform actions
    if key in [ord("q"), ord("Q")]:
        break
    elif key in [ord("p"), ord("P")]:
        mode = "Pen"
    elif key in [ord("e"), ord("E")]:
        mode = "Eraser"
    elif key in [ord("s"), ord("S")]:
        path = r"C:\Users\bhava\OneDrive\Desktop\computer vision\OpenCV_images{}.jpg".format(i)
        cv2.imwrite(path, white)
        i += 1
        mode = f"Saved: {path}"
        print(mode)
    elif key in [ord("d"), ord("D")]:
        white[:] = 255
        mode = "Deleted"

cv2.destroyAllWindows()